# Read seizure onsets from "all szrs" html file and output them to a csv file

In [16]:
import pandas as pd
import os
import numpy as np
from datetime import datetime
import pickle

In [3]:
sub='1146'
#metadata_dir='/Users/davidgroppe/PycharmProjects/SZR_ANT/EU_METADATA'
metadata_dir='/Users/davidgroppe/GIT/OCSVM_EDMSE/matlab/EU/metadata'
htable=pd.read_html(os.path.join(metadata_dir,'all_szrs_FR_'+sub+'.html'))
print('%d tables read' % len(htable))
print('This code expects 4 tables.')

4 tables read
This code expects 4 tables.


In [6]:
# FIRST TABLE IS CLINICAL SZRS
# Convert clinical onset/offsets to datetime
n_clinical_szr=htable[0].shape[0]
print('%d clinical szrs' % n_clinical_szr)
szr_onset_sec=list()
szr_offset_sec=list()
szr_onset_str=list()
szr_offset_str=list()
szr_type=list()

#milestone=datetime(1970,1,1)
milestone=datetime(2000,1,1) # Arbirary date via which to convert times into seconds
#milestone=datetime(2009,6,21)

for a in range(n_clinical_szr):
    szr_type.append('Clinical')
    
    print()
    # Convert onset to seconds since milestone
    temp_lst=htable[0].iloc[a,2].split(' ')
    temp_lst[1]=temp_lst[1].replace("'","20")
    print('Onset: '+temp_lst[1]+' '+temp_lst[2])
    hr_splt=temp_lst[2].split(':')
    temp_dt=datetime.strptime(temp_lst[1],'%d.%m.%Y')
    ttl_sec=(temp_dt-milestone).total_seconds()
    ttl_sec+=int(hr_splt[0])*3600+int(hr_splt[1])*60+float(hr_splt[2])
    szr_onset_sec.append(ttl_sec)
    szr_onset_str.append(temp_lst[1]+' '+temp_lst[2])
    
    # Convert offset to seconds since milestone
    temp_lst=htable[0].iloc[a,3].split(' ')
    temp_lst[1]=temp_lst[1].replace("'","20")
    print('Offset: '+temp_lst[1]+' '+temp_lst[2])
    hr_splt=temp_lst[2].split(':')
    temp_dt=datetime.strptime(temp_lst[1],'%d.%m.%Y')
    ttl_sec=(temp_dt-milestone).total_seconds()
    ttl_sec+=int(hr_splt[0])*3600+int(hr_splt[1])*60+float(hr_splt[2])
    szr_offset_sec.append(ttl_sec)
    szr_offset_str.append(temp_lst[1]+' '+temp_lst[2])

    df=szr_offset_sec[a]-szr_onset_sec[a]
    mnts=np.floor(df/60)
    scs=df-mnts*60
    print('Duration: %d minute(s) %f sec' % (mnts,scs))
    
    
# THIRD TABLE IS CLINICAL SZRS
# Convert SUBclinical onset/offsets to datetime
n_subclinical_szr=htable[2].shape[0]-1 # subtract 1 because last row is a total
print('%d subclinical szrs' % n_subclinical_szr)

for a in range(n_subclinical_szr):
    szr_type.append('Subclinical')
        
    print()
    # Convert onset to seconds since milestone
    temp_lst=htable[2].iloc[a,1].split(' ')
    temp_lst[0]=temp_lst[0].replace("'","20")
    print('Onset: '+temp_lst[0]+' '+temp_lst[1])
    hr_splt=temp_lst[1].split(':')
    temp_dt=datetime.strptime(temp_lst[0],'%d.%m.%Y')
    ttl_sec=(temp_dt-milestone).total_seconds()
    ttl_sec+=int(hr_splt[0])*3600+int(hr_splt[1])*60+float(hr_splt[2])
    #print(ttl_sec)
    szr_onset_sec.append(ttl_sec)
    szr_onset_str.append(temp_lst[0]+' '+temp_lst[1])
    
    # Convert offset to seconds since milestone
    temp_hr=htable[2].iloc[a,2]
    print('Offset: '+temp_lst[0]+' '+temp_hr)
    hr_splt=temp_hr.split(':')
    ttl_sec=(temp_dt-milestone).total_seconds()
    ttl_sec+=int(hr_splt[0])*3600+int(hr_splt[1])*60+float(hr_splt[2])
    szr_offset_sec.append(ttl_sec)
    szr_offset_str.append(temp_lst[0]+' '+temp_hr)

    df=szr_offset_sec[a]-szr_onset_sec[a]
    mnts=np.floor(df/60)
    scs=df-mnts*60
    print('Duration: %d minute(s) %f sec' % (mnts,scs))


9 clinical szrs

Onset: 21.06.2009 11:52:24.783203
Offset: 21.06.2009 11:54:26.308594
Duration: 2 minute(s) 1.525391 sec

Onset: 21.06.2009 13:56:49.808594
Offset: 21.06.2009 13:58:43.708008
Duration: 1 minute(s) 53.899414 sec

Onset: 22.06.2009 10:33:35.216797
Offset: 22.06.2009 10:35:32.900391
Duration: 1 minute(s) 57.683594 sec

Onset: 22.06.2009 14:58:49.866211
Offset: 22.06.2009 15:00:51.375000
Duration: 2 minute(s) 1.508789 sec

Onset: 23.06.2009 10:13:36.033203
Offset: 23.06.2009 10:16:13.383789
Duration: 2 minute(s) 37.350586 sec

Onset: 23.06.2009 11:45:01.008789
Offset: 23.06.2009 11:47:03.908203
Duration: 2 minute(s) 2.899414 sec

Onset: 23.06.2009 14:25:52.366211
Offset: 23.06.2009 14:27:52.616211
Duration: 2 minute(s) 0.250000 sec

Onset: 23.06.2009 16:09:33.741211
Offset: 23.06.2009 16:11:57.724609
Duration: 2 minute(s) 23.983398 sec

Onset: 24.06.2009 14:00:30.941406
Offset: 24.06.2009 14:01:43.983398
Duration: 1 minute(s) 13.041992 sec
22 subclinical szrs

Onset: 18.06.

In [7]:
# Create DataFrame
szr_on_off_df=pd.DataFrame({'SzrType': szr_type,
                           'SzrOnsetSec': szr_onset_sec,
                           'SzrOffsetSec': szr_offset_sec,
                           'SzrOnsetStr': szr_onset_str,
                           'SzrOffsetStr': szr_offset_str})

In [12]:
#szr_on_off_df.sort('SzrOnsetSec');
#szr_on_off_df.sort_values(by, axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
szr_on_off_df.sort_values('SzrOnsetSec',inplace=True)

In [13]:
szr_on_off_df.head()

,SzrOffsetSec,SzrOffsetStr,SzrOnsetSec,SzrOnsetStr,SzrType
9,298645408.0,18.06.2009 13:03:28,298645370.0,18.06.2009 13:02:50,Subclinical
10,298650199.0,18.06.2009 14:23:19,298650166.0,18.06.2009 14:22:46,Subclinical
11,298656769.0,18.06.2009 16:12:49,298656728.0,18.06.2009 16:12:08,Subclinical
12,298657941.0,18.06.2009 16:32:21,298657911.0,18.06.2009 16:31:51,Subclinical
13,298667569.0,18.06.2009 19:12:49,298667532.0,18.06.2009 19:12:12,Subclinical


In [15]:
# Output to csv
# out_path='/Users/davidgroppe/PycharmProjects/SZR_ANT/EU_METADATA'
# out_fname=os.path.join(out_path,'szr_on_off_FR_'+str(sub)+'.csv')
# print('Saving file to:')
# print(out_fname)
# szr_on_off_df.to_csv(out_fname)

Saving file to:
/Users/davidgroppe/PycharmProjects/SZR_ANT/EU_METADATA/szr_on_off_FR_1146.csv


In [19]:
# Output to pkl
out_path='/Users/davidgroppe/PycharmProjects/SZR_ANT/EU_METADATA'
out_fname=os.path.join(out_path,'szr_on_off_FR_'+str(sub)+'.pkl')
print('Saving file to:')
print(out_fname)
#pickle.dump(args,open('args.pickle','wb'))
pickle.dump(szr_on_off_df,open(out_fname,'wb'))

Saving file to:
/Users/davidgroppe/PycharmProjects/SZR_ANT/EU_METADATA/szr_on_off_FR_1146.pkl
